
**Trabalho pós facef - Caio Santos Freitas**


Essa atividade tem como objetivo descobrir um pouco mais sobre o cliente e traçar um perfil do mesmo. Visto que temos as informações pessoais do cliemte em um arquivo chamado "cliente_fake.csv", e os dados transacionais desse cliente no arquivo chamado "venda_fake". E idéia do exercicio é a manipulação dos dados de forma a fazer agregações, join nos dados, podendo ser necessário o uso de UDFs, gerando um Dataframe Final:

#### O Dataframe final deve estar agrupado por cliente (cliente_id), e conter as seguintes informações:

- cliente_id - O identificador do cliente
- idade - Calcular a idade do cliente (valor inteiro)
- recencia - Calcular a quantos meses faz que o cliente não compra (valor inteiro)
- qt_total_compras - Quantidade total de compras (pedidos distintos)
- vr_total_compras = Valor total de compras do cliente
- qt_total_itens - Quantidade total de itens comprados pelo cliente (produto_id distintos)
- qt_max_parcelas - Quantidade máxima de parcelas que o cliente já utilizou
- perfil_cliente - Deve mostrar o perfil de compra desse cliente, se compra somente "loja" ou somente "online", ou em ambos "multicanal";

OBS: O dataset "venda_fake" está a nível de itens. Pode haver mais de um item por pedido, ito é, o numero do pedido pode estar duplicado nodataset. 


In [1]:
from google.colab import files
uploaded = files.upload()

Saving cliente_fake.csv to cliente_fake.csv
Saving venda_fake.csv to venda_fake.csv


In [2]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 66kB/s 
     |████████████████████████████████| 204kB 46.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=668e4e48dcae08202b22376a4cafabfba5959f59a2cbae59b93121ecdbae6219
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [3]:
# coding: utf-8
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
    .appName('ConhecendoCliente') \
    .getOrCreate()

#### Dataset de cliente

In [5]:
df_cliente = spark.read \
    .option("delimiter", "|") \
    .csv('cliente_fake.csv', header=True)

df_cliente.show()

+--------------+----------+-------------+
|          nome|cliente_id|dt_nascimento|
+--------------+----------+-------------+
|Cliente 100013|    100013|   1964-06-21|
|Cliente 100014|    100014|   1964-05-24|
|Cliente 100015|    100015|   1946-10-18|
|Cliente 100016|    100016|   1972-03-24|
|Cliente 100017|    100017|   1982-01-19|
|Cliente 100018|    100018|   1983-04-26|
|Cliente 100019|    100019|   1971-09-09|
|Cliente 100020|    100020|   1989-06-18|
|Cliente 100007|    100007|   1976-11-04|
|Cliente 100008|    100008|   1980-12-01|
|Cliente 100009|    100009|   1981-03-05|
|Cliente 100010|    100010|   1984-03-20|
|Cliente 100011|    100011|   1977-11-22|
|Cliente 100012|    100012|   1974-02-04|
|Cliente 100001|    100001|   2009-04-16|
|Cliente 100002|    100002|   1974-08-29|
|Cliente 100003|    100003|   1981-12-05|
|Cliente 100004|    100004|   1968-08-08|
|Cliente 100005|    100005|   1982-11-29|
|Cliente 100006|    100006|   1948-04-01|
+--------------+----------+-------

#### Dataset de pedido

In [6]:
df_venda = spark.read \
    .option("delimiter", "|") \
    .csv('venda_fake.csv', header=True)

df_venda.show()

+--------------+----------+--------------------+----------+-----------+--------------+------------+-------+--------+
|     pedido_id|cliente_id|           dt_pedido|produto_id|qt_parcelas|   forma_pagto|vr_unit_item|qt_item|tp_canal|
+--------------+----------+--------------------+----------+-----------+--------------+------------+-------+--------+
|10000720200217|    100007|2020-02-17T00:00:...|   6253644|         20|cartao credito|         478|      1|    loja|
|10000720200217|    100007|2020-02-17T00:00:...|   5313408|         20|cartao credito|      1357.2|      1|    loja|
|10000720200217|    100007|2020-02-17T00:00:...|   6253140|         20|cartao credito|         372|      1|    loja|
|10000720200625|    100007|2020-06-25T00:00:...|   6858780|          7|cartao credito|       317.8|      1|    site|
|10000120200626|    100001|2020-06-26T00:00:...|   7560984|         16|cartao credito|     1399.04|      1|    loja|
|10000920200614|    100009|2020-06-14T00:00:...|   7571280|     

In [60]:
import pyspark.sql.functions as sf

idade = df_cliente \
  .select(
      sf.col('cliente_id'),
      sf.col('dt_nascimento')) \
  .withColumn('idade', (sf.year(sf.current_date()) - sf.year('dt_nascimento')))

idade = idade \
    .select(
      sf.col('cliente_id'),
      sf.col('idade'))
    
recencia = df_venda \
  .select(
      sf.col('cliente_id'),
      sf.col('dt_pedido').cast('timestamp')) \
  .withColumn('recencia', sf.round(sf.months_between(sf.current_date(), 'dt_pedido'))) \
  .groupBy('cliente_id') \
  .max('recencia') \
  .withColumnRenamed('max(recencia)', 'recencia')

qt_total_compras = df_venda \
  .select(
      sf.col('cliente_id'),
      sf.col('pedido_id')) \
  .distinct() \
  .groupBy('cliente_id') \
  .count() \
  .withColumnRenamed('count', 'qt_total_compras')

vr_total_compras = df_venda \
  .select(
      sf.col('cliente_id'),
      sf.col('vr_unit_item').cast('numeric(8,2)'),
      sf.col('qt_item').cast('integer')) \
  .withColumn('vr_total_items', 
              sf.col('vr_unit_item').cast('numeric(8,2)') * \
              sf.col('qt_item').cast('integer')) \
  .groupBy('cliente_id') \
  .sum('vr_total_items') \
  .withColumnRenamed('sum(vr_total_items)', 'vr_total_compras')

qt_total_itens = df_venda \
  .select(
      sf.col('cliente_id'),
      sf.col('produto_id')) \
  .distinct() \
  .groupBy('cliente_id') \
  .count() \
  .withColumnRenamed('count', 'qt_total_itens')

qt_max_parcelas = df_venda \
  .select(
      sf.col('cliente_id'),
      sf.col('qt_parcelas').cast('integer')) \
  .distinct() \
  .groupBy('cliente_id') \
  .max('qt_parcelas') \
  .withColumnRenamed('max(qt_parcelas)', 'qt_max_parcelas')

perfil_cliente_nome = df_venda \
  .select(
      sf.col('tp_canal'),
      sf.col('cliente_id')) \
  .distinct() \
  .groupBy('cliente_id') \
  .agg(sf.max("tp_canal").alias("tp_canal"))

perfil_cliente_qt = df_venda \
  .select(
      sf.col('tp_canal'),
      sf.col('cliente_id')) \
  .distinct() \
  .groupBy('cliente_id') \
  .count() \
  .withColumnRenamed('count', 'qt_canal')
  
perfil_cliente = perfil_cliente_nome.join(perfil_cliente_qt, 'cliente_id') \
  .withColumn('perfil_cliente', sf.when(sf.col('qt_canal') == 1, sf.col('tp_canal'))
  .otherwise("multicanal"))

perfil_cliente = perfil_cliente \
  .select(
    sf.col('cliente_id'),
    sf.col('perfil_cliente'))

output = idade.join(recencia,'cliente_id') \
  .join(qt_total_compras,'cliente_id') \
  .join(vr_total_compras,'cliente_id') \
  .join(qt_total_itens,'cliente_id') \
  .join(qt_max_parcelas,'cliente_id') \
  .join(perfil_cliente,'cliente_id')
  
output.show()



+----------+-----+--------+----------------+----------------+--------------+---------------+--------------+
|cliente_id|idade|recencia|qt_total_compras|vr_total_compras|qt_total_itens|qt_max_parcelas|perfil_cliente|
+----------+-----+--------+----------------+----------------+--------------+---------------+--------------+
|    100010|   36|     9.0|               1|          898.97|             1|              9|          loja|
|    100014|   56|     8.0|               2|         1746.01|             3|             11|          loja|
|    100008|   40|     4.0|               1|         1399.86|             1|             14|          loja|
|    100006|   72|     3.0|               1|         3170.72|             3|              9|          loja|
|    100005|   38|     9.0|               2|         3207.28|             2|             15|          loja|
|    100007|   44|     8.0|               2|         2525.00|             4|             20|    multicanal|
|    100020|   31|     3.0| 